In [1]:
# Install all necessary packages
!pip install -q yt-dlp librosa soundfile torchaudio demucs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resoluti

In [ ]:
# Download YouTube audio and save as mp3 with video title as filename
import os

youtube_url = "https://youtu.be/b0Tk-FoiX_0?feature=shared"  # ⬅️ Replace with your link

!yt-dlp -x --audio-format mp3 -o "%(title)s.%(ext)s" "{youtube_url}"


[youtube] Extracting URL: https://youtu.be/b0Tk-FoiX_0?feature=shared
[youtube] b0Tk-FoiX_0: Downloading webpage
[youtube] b0Tk-FoiX_0: Downloading tv client config
[youtube] b0Tk-FoiX_0: Downloading player 20830619
[youtube] b0Tk-FoiX_0: Downloading tv player API JSON
[youtube] b0Tk-FoiX_0: Downloading ios player API JSON
[youtube] b0Tk-FoiX_0: Downloading m3u8 information
[info] b0Tk-FoiX_0: Downloading 1 format(s): 251
[download] Destination: Small Axe (1973) - Bob Marley & The Wailers.webm
[download] 100% of    3.70MiB in 00:00:00 at 22.29MiB/s
[ExtractAudio] Destination: Small Axe (1973) - Bob Marley & The Wailers.mp3
Deleting original file Small Axe (1973) - Bob Marley & The Wailers.webm (pass -k to keep)


In [ ]:
import os
import librosa
import soundfile as sf
import glob
import re

# Find the downloaded MP3 file
mp3_files = glob.glob("*.mp3")
assert mp3_files, "No MP3 files found!"
input_audio = mp3_files[0]

# Clean "NA - " prefix if it exists
if input_audio.startswith("NA - "):
    cleaned_name = input_audio.replace("NA - ", "", 1)
    os.rename(input_audio, cleaned_name)
    print(f"Cleaned filename: {cleaned_name}")
    input_audio = cleaned_name  # Update variable

# Load audio
y, sr = librosa.load(input_audio, sr=None)

# Limit to first 2 minutes for faster analysis
max_duration = 120  # seconds
y = y[:sr * max_duration]

# BPM detection
tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
bpm = int(tempo.item() if hasattr(tempo, "item") else tempo)

# Key estimation
chroma = librosa.feature.chroma_cens(y=y, sr=sr)
chroma_mean = chroma.mean(axis=1)
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
key_index = chroma_mean.argmax()
estimated_key = note_names[key_index]

# Rename file with key + bpm if not already tagged
base, ext = os.path.splitext(input_audio)
tag = f"[{estimated_key} {bpm}]"
if tag not in base:
    new_filename = f"{base} {tag}{ext}"
    os.rename(input_audio, new_filename)
    print(f"Renamed to: {new_filename}")
else:
    new_filename = input_audio
    print(f"Already tagged: {new_filename}")


Renamed to: Small Axe (1973) - Bob Marley & The Wailers [A 181].mp3


In [ ]:
from demucs.pretrained import get_model
from demucs.apply import apply_model
import torchaudio
import torch
import os
import glob

# Locate the renamed file
mp3_files = glob.glob("*.mp3")
assert mp3_files, "No renamed MP3 found!"
input_audio = mp3_files[0]

# Load and resample (keep stereo!)
waveform, sr = torchaudio.load(input_audio)
waveform = torchaudio.functional.resample(waveform, sr, 44100)
waveform = waveform.unsqueeze(0)  # Add batch dim: [1, channels, time]
sr = 44100

# Load Demucs model
model = get_model(name='htdemucs')
model.eval()

# Apply model
with torch.no_grad():
    sources = apply_model(model, waveform, device='cpu')[0]

# Save separated stems
output_dir = "demucs_output"
os.makedirs(output_dir, exist_ok=True)

for name, source in zip(model.sources, sources):
    path = os.path.join(output_dir, f"{name}.wav")
    torchaudio.save(path, source.cpu(), sr)
    print(f"Saved: {path}")


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:01<00:00, 81.9MB/s]


Saved: demucs_output/drums.wav
Saved: demucs_output/bass.wav
Saved: demucs_output/other.wav
Saved: demucs_output/vocals.wav
